<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [1]:
!pip install pyspark
!pip install findspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 1.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 28.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=3eb78d436514dab4b6a7e879efe21ab38d9c014efa38ca85394166161225b3e6
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/06/51/98/f7a41aad64c08302d6c26c90650e713c3dfeb5cdec4946db00
Successfully built pyspark


In [26]:
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 25.5 MB/s eta 0:00:00


In [ ]:
# Start session

In [2]:
import findspark
findspark.init()

In [21]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import requests
from pyspark.sql.functions import col
import tarfile
from io import BytesIO

In [4]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("webserver log").getOrCreate()

23/06/05 07:26:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [24]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [6]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv"
filename = "searchterms.csv"

response = requests.get(url)
response.raise_for_status()

with open(filename, "wb") as file:
    file.write(response.content)

print("Dataset downloaded successfully!")

Dataset downloaded successfully!


In [ ]:
# Load the csv into a spark dataframe

In [7]:
csv_path = "searchterms.csv"

# Read the CSV file into a DataFrame
df = spark.read.format("csv").option("header", "true").load(csv_path)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [9]:
# Number of rows
num_rows = df.count()

# Number of columns
num_cols = len(df.columns)

print("Number of rows:", num_rows)
print("Number of columns:", num_cols)

Number of rows: 10000
Number of columns: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [10]:
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [11]:
df.printSchema()

root
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- year: string (nullable = true)
 |-- searchterm: string (nullable = true)



In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [12]:
search_term = "gaming laptop"
count = df.filter(df.searchterm == search_term).count()
print("Number of times 'gaming laptop' was searched:", count)

Number of times 'gaming laptop' was searched: 499


In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [15]:
# Group by searchterm and count the occurrences
search_term_counts = df.groupBy("searchterm").count()

# Order by count in descending order
search_term_counts = search_term_counts.orderBy(col("count").desc())

# Print the top 5 search terms
top_5_search_terms = search_term_counts.limit(5)
top_5_search_terms.show()


[Stage 9:====================================================>  (192 + 8) / 200]

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [18]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz"
filename = "model.tar.gz"

response = requests.get(url)
response.raise_for_status()

with open(filename, "wb") as file:
    file.write(response.content)

print("Model downloaded successfully!")

Model downloaded successfully!


In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [31]:
tar_file_path = "model.tar.gz"
# Directory to extract the contents of the tar file
extracted_dir = "."

# Extract the tar file
with tarfile.open(tar_file_path, "r:gz") as tar:
    tar.extractall(extracted_dir)

# Start a Spark session

# Load the Parquet files into a DataFrame
model_df = spark.read.parquet("sales_prediction.model/data/*.parquet")

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [34]:
intercept = model_df.select("intercept").first()[0]
coefficients = model_df.select("coefficients").first()[0][0]

# Year for prediction
prediction_year = 2023

# Calculate the predicted sales for the year 2023
predicted_sales = intercept + (coefficients * prediction_year)

# Display the predicted sales for the year 2023
print("Predicted sales for 2023:", predicted_sales)

Predicted sales for 2023: 175.16564294006457
